# HW2 for CS 135 in Fall 2023

[Main assignment instructions here](https://www.cs.tufts.edu/cs/135/2024f/Assignments/hw2.html)

This notebook is specifically for the report related to Problem 2

# Problem 2: Binary Classifier for Cancer-Risk Screening 

### Note: 
* You will need to finish **Code Task** for **Problem 1** before working on this notebook. 
* Your outputs, including tables and figures, don't need to be exactly the same as our sample outputs, but we do expect something as clear.

In [1]:
import os
import numpy as np
import pandas as pd

import sklearn.linear_model
import sklearn.metrics

In [ ]:
# import plotting libraries
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', style='whitegrid', font_scale=1.25)

# autoload changes in other files, so you don't have to restart the Jupyter kernel each time you make a change to the imported code.
%load_ext autoreload
%autoreload 2

# Import student-edited code 

Remember, you should have *completed* the Code Tasks for Problem 1 first. Note that this file needs to be in the same directory/folder as `binary_metrics.py` in order to function. 

In [ ]:
if not os.path.exists('binary_metrics.py'):
    raise ImportError("CANNOT FIND binary_metrics.py. Make sure you run this notebook in same directory as your .py files.")

In [ ]:
from binary_metrics import (
    calc_ACC, calc_TPR, calc_PPV)

# Import helper code (will work as provided, no edits needed)

In [ ]:
import threshold_selection

In [ ]:
from confusion_matrix import calc_confusion_matrix_for_probas_and_threshold

# Provided function for computing mean binary cross entropy

Here, we provide a *completed* function you can use as-is for Problem 1.

Remember, we want the *base-2* cross entropy:

$$
BCE(y, p) = - y \log_2 p - (1-y) \log_2(1-p)
$$

In [ ]:
def calc_mean_binary_cross_entropy_from_probas(ytrue_N, yproba1_N):
    ''' Compute mean binary cross entropy
    
    Args
    ----
    ytrue_N : 1D array, size (n_examples,) = (N,)
    yproba1_N : 1D array, size (n_examples,) = (N,)
    
    Returns
    -------
    mean_bce : float
        mean binary cross entropy across all N examples
    '''
    return sklearn.metrics.log_loss(ytrue_N, yproba1_N, labels=[0,1]) / np.log(2.0)

In [ ]:
# Check that BCE loss is high if true class is 1 but probability is low
calc_mean_binary_cross_entropy_from_probas([1.], [0.01])

In [ ]:
# Check that BCE loss is exactly 1 if true class is 1 but probability is 0.5
calc_mean_binary_cross_entropy_from_probas([1.], [0.5])

In [ ]:
# Check that BCE loss is close to zero if true class is 1 but probability is 0.99
calc_mean_binary_cross_entropy_from_probas([1.], [0.99])

# Load cancer dataset

In [ ]:
# Make sure you have downloaded data and your directory is correct
DATA_DIR = os.path.join('../data_cancer') 

In [ ]:
# Load 3 feature version of x arrays
x_tr_M3 = np.loadtxt(os.path.join(DATA_DIR, 'x_train.csv'), delimiter=',', skiprows=1)
x_va_N3 = np.loadtxt(os.path.join(DATA_DIR, 'x_valid.csv'), delimiter=',', skiprows=1)
x_te_N3 = np.loadtxt(os.path.join(DATA_DIR, 'x_test.csv'), delimiter=',', skiprows=1)

for label, x in [('train', x_tr_M3), ('valid', x_va_N3), ('test', x_te_N3)]:
    print("Loaded %6s : shape %s" % (label, x.shape))

In [ ]:
N_shape = x_va_N3.shape
M_shape = x_tr_M3.shape

N = N_shape[0]
M = M_shape[0]

## Implementation Step 1A: Data Exploration

**TODO** : Load outcomes **y** arrays

In [ ]:
# Load outcomes y arrays
# Hint: Follow the way we import the x arrays above.
y_tr_M = np.hstack([np.zeros(M//2), np.ones(M//2)]) # TODO fixme
y_va_N = np.hstack([np.zeros(N//2), np.ones(N//2)]) # TODO fixme
y_te_N = np.hstack([np.zeros(N//2), np.ones(N//2)]) # TODO fixme

In [ ]:
# Load 2 feature version of x arrays
x_tr_M2 = x_tr_M3[:, :2].copy()
x_va_N2 = x_va_N3[:, :2].copy()
x_te_N2 = x_te_N3[:, :2].copy()

### Table 1 

Create and print a pandas table summarizing some basic properties of the provided training set, validation set, and test set:

* Row 1 'total count': how many total examples are in each set?
* Row 2 'positive label count': how many examples have a positive label (means cancer)?
* Row 3 'fraction positive' : what fraction (between 0 and 1) of the examples have cancer?

Your result should be 3 by 3 containing 'total count', 'positive label count' and 'fraction positive' of training, valid and test sets. An example of the output looks like this (note that the numbers in this table are examples, not the numbers that should be in your table):

|                               | train  |valid    |test|
|:-|-:|-:|-:|
|num. total examples            |567.000  |123.000  |123.000
|num. positive examples         |56.000   |23.000   |23.000
|fraction of positive examples  |0.123    |0.123    |0.123

All results should **keep 3 digits**. We set a pandas display option to ensure that below.

**TODO**: make a pandas dataframe with the correct data properties

In [ ]:
table1_df = pd.DataFrame() # TODO fixme
table1_df.index = ['num. total examples', 'num. positive examples', 'fraction of positive examples']

In [ ]:
pd.set_option('display.precision', 3)
print(table1_df)

##  Implementation Step 1B: The predict-0-always baseline

**TODO**: predict zero for all test data points

In [ ]:
baseline_yhat_te_N = np.random.RandomState(0).randint(low=0, high=2, size=N) # TODO fixme
print(baseline_yhat_te_N)

In [ ]:
print(y_te_N)

In [ ]:
print(calc_confusion_matrix_for_probas_and_threshold(y_te_N, baseline_yhat_te_N, 0.5))

**TODO** Use the printed information from the previous code cell to calculate the accuracy of baseline. 

Keep 3 digits in your PDF report for short Answer 1a.

In [ ]:
baseline_acc = -1.0 # TODO: calculate the accuracy of baseline.
print("Baseline has accuracy: ", baseline_acc)

#### Short Answer 1a

What *accuracy* does the "predict-0-always" classifier get on the *test* set (report to 3 decimal places)?
(You should see a pretty high number). Why isn't this classifier "good enough" to use in our screening task?

## Implementation Step 1C : Logistic Regression with F=2 dataset

**TODO**: Complete each line marked TODO fixme in the codeblock below

In [ ]:
C_grid = np.logspace(-9, 6, 31)

# We will fit a separate logistic regression for each C value in the C_grid
# And store that classifier's performance metrics (lower is better)
# So we can compare and select the best C in the future steps.

model_F2_list = list()

# Allocate lists for storing BCE metrics
tr_bce_list = list()
va_bce_list = list()

# Allocate lists for storing ERROR RATE metrics
tr_err_list = list()
va_err_list = list()

# Remember, we justified BCE for training our classifier by saying
# it provides an *upper bound* on the error rate.

# Loop over C values, fit models, record metrics
for C in C_grid:
    # TODO: Follow the instruction in HW2 and train the model lr_F2
    # Part a: Initialize a LogisticRegression classifier with desired C value
    # Part b: train the model with the 2-feature dataset
    lr_F2 = sklearn.linear_model.LogisticRegression() # TODO fixme
    lr_F2.fit(x_tr_M2[::5], y_tr_M[::5]) # TODO fixme
    
    model_F2_list.append(lr_F2)
    
    yproba1_tr_M = lr_F2.predict_proba(x_tr_M2)[:,1] # The probability of predicting class 1 on the training set
    yproba1_va_N = lr_F2.predict_proba(x_va_N2)[:,1] # The probability of predicting class 1 on the validation set
    
    # Compute error rate aka zero-one loss
    my_tr_err = sklearn.metrics.zero_one_loss(y_tr_M, yproba1_tr_M >= 0.5)
    my_va_err = sklearn.metrics.zero_one_loss(y_va_N, yproba1_va_N >= 0.5)
    tr_err_list.append(my_tr_err)
    va_err_list.append(my_va_err)
    
    # TODO: using the calc_mean_binary_cross_entropy_from_probas() function from above:
    # Part c: calculate the binary cross entropy (bce) on the training set
    # Part d: calculate the binary cross entropy (bce) on the validation set
    my_tr_bce = my_tr_err + 0.1 # TODO fixme
    my_va_bce = my_va_err + 0.15 # TODO fixme
    # Save bce for future selection on Models.
    tr_bce_list.append(my_tr_bce) 
    va_bce_list.append(my_va_bce) 

In [ ]:
plt.plot(np.log10(C_grid), tr_bce_list, 'bs-', label='train BCE')
plt.plot(np.log10(C_grid), va_bce_list, 'rs-', label='valid BCE')

plt.plot(np.log10(C_grid), tr_err_list, 'b:', label='train err')
plt.plot(np.log10(C_grid), va_err_list, 'r:', label='valid err')

plt.ylabel('error')
plt.xlabel("$\log_{10} C$");
plt.legend(bbox_to_anchor=(1.5, 0.5)) # make legend outside plot
plt.ylim([0, 0.7]);

### Check Point:

If your code is correct, you should produce exactly the same figure when you run the cell above as:

**"Checkpoint1C.jpg"** in the same folder as your starter code.
![image.png](Checkpoint1C.JPG)

In [ ]:
print("Searched over these C values:")
print(C_grid)

print("Recorded these BCE loss values on val set")
print(va_bce_list)

**TODO** Find the best C with the samllest cross entropy loss on the validation set.

In [ ]:
# Find the best C with the smallest cross entropy loss on the validation set
best_C__lrF2 = C_grid[-1] # TODO fixme

print("Best C value for F2 model:")
print(best_C__lrF2)

**TODO** Load the model that was rated 'best'

In [ ]:
best_lrF2 = model_F2_list[-1] # TODO fixme

print("Best model has coefficient values:")
print(best_lrF2.coef_)

In [ ]:
# Output the prediction of your best model for 2-feature data on the training, valid, and test set. 
# Return the posibility of predicting true
# We'll use them for the ROC curve
bestlrF2_yproba1_tr_M = best_lrF2.predict_proba(x_tr_M2)[:,1]
bestlrF2_yproba1_va_N = best_lrF2.predict_proba(x_va_N2)[:,1]
bestlrF2_yproba1_te_N = best_lrF2.predict_proba(x_te_N2)[:,1]

## Implementation Step 1D : Logistic Regression with F=3 dataset
**TODO**: 
* Repeat Step 1C for 3-feature Dataset to find the best C. 

**TODO**:
* Select the best C and retrieve the best model. You should name the model as "**best_lrF3**".

**TODO**:
* Return the posibility of predicting true on training, validation and test set.

In [ ]:
# TODO: Return the posibility of predicting true on training, validation and test set.
# Hint: follow what's done in the last code block of Step 1C.

prng = np.random.RandomState(101)
bestlrF3_yproba1_tr_M = prng.rand(M) # TODO fixme
bestlrF3_yproba1_va_N = prng.rand(N) # TODO fixme
bestlrF3_yproba1_te_N = prng.rand(N) # TODO fixme

## Implementation Step 1E : Comparing Models using ROC Analysis
**TODO**:
Follow the instruction to produce the figure for your report

In [ ]:
# Follow the instruction of HW2 and produce Figure 1 for your report:
plt.subplots(nrows=1, ncols=1, figsize=(5,5));

# TODO Use provided data and predictions on the Validation set
# Produce the ROC Curve utilizing `sklearn.metrics.roc_curve' within plt defined above.
# To start, read the official Document and examples of 'sklearn.metrics.roc_curve'. 

# Read HW2 instructions carefully for plot style (line type, line color, etc.)

plt.title("ROC on Validation Set");
plt.xlabel('false positive rate');
plt.ylabel('true positive rate');
plt.legend(loc='lower right');
B = 0.01
plt.xlim([0 - B, 1 + B]);
plt.ylim([0 - B, 1 + B]);

### Figure 1 for the report: Comparing models using ROC analysis
**TODO**:
Show the figure above in your report

#### Short Answer 1b

Compare the two models in terms of their ROC curves from Figure 1. Does one dominate the other in terms of overall performance across all thresholds, or are there some threshold regimes where the 2-feature model is preferred and other regimes where the 3-feature model is preferred? Which model do you recommend for the task at hand?

## Selecting a decision threshold

Now that we've compared models, we need to decide on a classification *threshold* to obtain a binary decision from probabilities. 

To get candidate threshold values, use the helper function `compute_perf_metrics_across_thresholds` in the starter code file [threshold_selection.py](https://github.com/tufts-ml-courses/cs135-23f-assignments/blob/main/hw2/threshold_selection.py).

#### Implementation Step 1F

For the classifier from 1D above (LR for 3-features), calculate performance metrics using the default threshold of `y_proba < 0.5`. Produce the confusion matrix and calculate the TPR and PPV. 
*Tip: Remember that we have implemented helper functions for you in `confusion_matrix.py`.*

TODO:

* Use F=3 LR model, Use default 0.5 threshold
* Produce the confusion matrix
* Return TPR and PPV on the test set

In [2]:
best_thr_default = 0.5
tpr = 0.0 # TODO fixme
ppv = 0.0 # TODO fixme

print(f"Default threshold (0.5): chosen thr = {best_thr_default :.4f}, tpr = {tpr :.4f}, ppv = {ppv :.4f},")

Default threshold (0.5): chosen thr = 0.5000, tpr = 0.0000, ppv = 0.0000,


### Implementation Step 1G

For the classifier from 1D above (LR for 3-features), compute performance metrics across all candidate thresholds on the validation set (use `compute_perf_metrics_across_thresholds`). Then, pick the threshold that *maximizes TPR while satisfying PPV >= 0.98* on the validation set.
If there's a tie for the maximum TPR, chose the threshold corresponding to a higher PPV. 

Remember, you pick this threshold based on the *validation* set, then later you'll evaluate it on the *test* set.

**TODO**: Finish the code block to get the best threshold.

In [ ]:
# Get the performance metrics across many thresholds
thresh_grid, va_perf_grid = threshold_selection.compute_perf_metrics_across_thresholds(
    y_va_N, bestlrF3_yproba1_va_N)

In [ ]:
# Find threshold that makes TPR as large as possible, while satisfying PPV >= 0.98
# TODO Find the the largest TPR while PPV >= 0.98

best_thr_tpr = 0.5 # TODO fixme
tpr1 = 0.0 # TODO fixme
ppv1 = 0.0 # TODO fixme

print(f"Figure2 column 2: chosen thr = {best_thr_tpr :.4f}, tpr = {tpr1 :.4f}, ppv = {ppv1 :.4f},")

**TODO**: Using the chosen **best_thr** above
* Produce the confusion matrix on the test set
* Return TPR and PPV on the test set

### Implementation Step 1H

Now using the same F=3 LR model, pick a threshold to maximize PPV s.t. TPR >= 0.98

TODO: 
Using a similar logistic from **Step 1F** above
* Choose threshold that makes PPV as large as possible, while satisfying TPR >= 0.98
* Produce the confusion matrix using the chosen threshold
* Return TPR and PPV on the test set using the chosen threshold

In [ ]:
# TODO Find the largest PPV while TPR >= 0.98
best_thr_ppv = 0.5 # TODO fixme
tpr2 = 0.0 # TODO fixme
ppv2 = 0.0 # TODO fixme

print("Figure2 column 3: chosen thr = %.4f, tpr = %.4f, ppv = %.4f" % (best_thr_ppv, tpr2, ppv2))

#### Short Answer 1c

By carefully reading the confusion matrices above, for *each thresholding strategy* how many subjects in the test set are saved from unnecessary biopsies that would be done in current practice?

*Hint*: You can assume that currently, the hospital would have done a biopsy on every patient in the test set. Your goal is to build a classifier that improves on this practice. 

#### Short Answer 1d

Among the 3 possible thresholding strategies, which strategy best meets the stated goals of stakeholders in this screening task: avoid life-threatening mistakes whenever possible, while also eliminating unnecessary biopsies?
What fraction of current biopsies might be avoided if this strategy was adopted by the hospital? 

*Hint*: You can also assume the test set is a reasonable representation of the true population of patients.

# Reflection

1. How many hours, approximatley, did this homework take you?
2. What did you feel was most confusing about this homework?
3. Did you recieve help on this homework from any classmates, stack overflow threads, AI tools, Youtube videos, etc. for this homework? If so, please list them here. 